# Build Topic Model
The present notebook will use the data collected in section 5.1 to compute and explore a topic model. In essence, a topic model consists of two sets of probability distributions. First, the probability that a *word* belongs to a certain topic (Topic/Term probability). Second, the probability of a *topic* to appear in a document (Document/Topic probability). These probabilities are never 0, that is, each word has some probability (even if very small) to be part of each topic and each topic has some probability (even if very small) to appear in a document.

A standard way to inspect a topic model is to look at the top-ten words of a topic (the ten words with the highest probability in each particular topic). Similarly, we can pull the ten most important documents for each topic (the documents in which this topic has the highest probability).

For a more thorough analysis we may create full probability tables: a topic/term probability table and a document/topic probability table. These tables give a fuller account of the model and will be used for the visualizations (5.3). 

In [4]:
import numpy as np
import pandas as pd
from gensim import corpora, models, utils
import gensim
import pickle

# Read in the texts
[For test purposes one may select only the first 100 documents. Remove the hashmark (#) from the first line of the following cell if you wish to do that]

In [5]:
pickled = 'output/data_for_topic_model.p'
df = pd.read_pickle(pickled)
texts = df['lemma']

# POS-filter
The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `[']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [6]:
posfilter = [']n', ']v', 'aj']
#include nouns, verbs, and adjectives, not numerals, prepositions or proper nouns
texts = [[word for word in text if word[-2:] in posfilter] for text in texts]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but those can also be filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis.

In [7]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v'
]
texts = [[word for word in text if word not in stoplist] for text in texts]


# Filter out texts that have too few words left
Identify texts that have less than 10 lemmas left and use that selection for the list `texts` and for the dataframe `df`. 

In [8]:
bo = [len(text)>9 for text in texts]
df = df[bo]
texts = [texts[i] for i in range(0, len(texts)) if bo[i]]

How many documents did we start with, and how many do we have left?

In [9]:
len(bo), len(df)

(4976, 3006)

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [10]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
## CHECK - is this done correctly?

In [11]:
corpus = [dictionary.doc2bow(doc) for doc in texts]

# Compute the Model

Set the seed, indicate the number of topics (default set to 10) and run the model.

The visualization (section 5.3) will fail if the number of topics is higher than 25. 

In [12]:
ntopics = int(input("Number of topics: ") or 10)
if ntopics > 25:
    ntopics = 25

Number of topics:  


In [13]:
seed = 15
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Training LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=50)

List the top 10 words and their probabilities in all topics. Note: the topic numbers here are not the ones used in the visualizations in 5.3! (The topics are the same, but not their numbers).

In [14]:
ldamodel.show_topics(ntopics, formatted = False)

[(0,
  [('qû[unit]n', 0.039634343),
   ('ilu[god]n', 0.02959446),
   ('šikaru[beer]n', 0.024415387),
   ('karānu[vine]n', 0.022837134),
   ('šamnu[oil]n', 0.020545267),
   ('kusāpu[bread]n', 0.017754514),
   ('zamāru[sing]v', 0.016015612),
   ('dišpu[honey]n', 0.014499569),
   ('nīqu[offering]n', 0.01421817),
   ('naqû[pour-(a-libation)]v', 0.014005929)]),
 (1,
  [('nišu[people]n', 0.06957513),
   ('imēru[unit]n', 0.06602787),
   ('eqlu[field]n', 0.057719853),
   ('immeru[sheep]n', 0.04232107),
   ('ikkaru[farmer]n', 0.041318834),
   ('alpu[ox]n', 0.03420751),
   ('kirû[garden]n', 0.03134715),
   ('sinništu[woman]n', 0.02788458),
   ('tarbaṣu[courtyard]n', 0.025301296),
   ('lawû[surround]v', 0.024567384)]),
 (2,
  [('rabû[big-one]n', 0.072569),
   ('pīhātu[responsibility]n', 0.039937504),
   ('ša-rēši[eunuch]n', 0.024973528),
   ('ša-qurbūti[close-follower]n', 0.024012562),
   ('ṣābu[people]n', 0.02039524),
   ('ēkallu[palace]n', 0.019527601),
   ('šaknu[appointee]n', 0.018716713),
  

# Document/Topic Probability
The function `get_document_topics()` will list the probability of the topics in a single document. In order to get all the topics set the argument `minimum_probability` to zero. 

In [15]:
ldamodel.get_document_topics(corpus[1], minimum_probability=0)

[(0, 0.044934507),
 (1, 0.0038660131),
 (2, 0.046452247),
 (3, 0.0038658625),
 (4, 0.0038665223),
 (5, 0.0038663503),
 (6, 0.087799765),
 (7, 0.09957184),
 (8, 0.0038660103),
 (9, 0.7019109)]

# Create Document/Topic Probability Table
A Document/Topiuc probability table is a table (DataFrame), where each row represents a document and each column a topuic. Each cell has the probability of a particular topic in a particular document. The sum of each row is 1 (probability distribution).

In order to create a full Document/Topic probability table we iterate over the entire corpus with the `get_document_topics()` function. This creates a list of lists (`list_of_doctopics`) where each list represents the probability of each topic in a document. The probability is represented in a tuple (topic_number, probability). The `list_of_probabilities` preserves only the probabilities. This list of lists is transformed into a DataFrame, whith as index the index of the original DataFrame with the tokenized data.  

In [16]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
d_t_df = pd.DataFrame(list_of_probabilities)
d_t_df = d_t_df.set_index(df.index)
d_t_df.head()

,0,1,2,3,4,5,6,7,8,9
P224378,0.009094,0.009094,0.009096,0.009096,0.009097,0.102053,0.009094,0.009095,0.009094,0.825185
P224382,0.044935,0.003866,0.046453,0.003866,0.003867,0.003866,0.087804,0.099572,0.003866,0.701906
P224383,0.003334,0.003334,0.320284,0.003334,0.003334,0.003335,0.003334,0.003334,0.003335,0.653042
P224386,0.006667,0.006667,0.006668,0.006668,0.006669,0.006667,0.006667,0.006669,0.156555,0.790102
P224388,0.006669,0.006671,0.098229,0.006670,0.006670,0.006670,0.006670,0.006670,0.006670,0.848412


We can use the above table to find the ten highest scoring documents per topic with the pandas function 'nlargest'.

In [17]:
new = pd.merge(df['designation'], d_t_df, left_index=True, right_index=True)
new

,designation,0,1,2,3,4,5,6,7,8,9
P224378,Take Over the Kingship!,0.009094,0.009094,0.009096,0.009096,0.009097,0.102053,0.009094,0.009095,0.009094,0.825185
P224382,Family Affairs,0.044935,0.003866,0.046453,0.003866,0.003867,0.003866,0.087804,0.099572,0.003866,0.701906
P224383,Šubrian King Protecting Deserters,0.003334,0.003334,0.320284,0.003334,0.003334,0.003335,0.003334,0.003334,0.003335,0.653042
P224386,Specialists Reviving the Land,0.006667,0.006667,0.006668,0.006668,0.006669,0.006667,0.006667,0.006669,0.156555,0.790102
P224388,Elamite King and the Men of Mukin-zeri,0.006669,0.006671,0.098229,0.006670,0.006670,0.006670,0.006670,0.006670,0.006670,0.848412
...,...,...,...,...,...,...,...,...,...,...,...
X900011,Reclaiming Runaway Servants,0.002942,0.002943,0.002942,0.002942,0.002942,0.002942,0.245046,0.002942,0.373325,0.361034
X900012,Receiving Tribute Horses; Constructing Kar-Šar...,0.002084,0.002084,0.477666,0.002084,0.002084,0.002084,0.002084,0.002084,0.053267,0.454479
X900013,Midas of Phrygia Seeks Detente,0.001164,0.001163,0.001164,0.001163,0.001164,0.001164,0.001164,0.001163,0.014563,0.976129
X900014,Report on a Festival and on Bull Colossi,0.002273,0.002273,0.979539,0.002273,0.002273,0.002274,0.002273,0.002273,0.002274,0.002274


In [18]:
doctop = []
for i in range(ntopics):
    t = new.nlargest(10, i)[['designation', i]]
    t['topic'] = i
    t = t.rename(columns = {i :'probability'})
    doctop.append(t)
doctop_df = pd.concat(doctop, axis=0)
doctop_df

,designation,probability,topic
P336282,Unplaced Fragment of the Text of No. 69,0.976921,0
P335833,"Aššur Temple Offerings, Day 10",0.974283,0
P282529,Rituals on [Shebat 23?],0.955194,0
P425166,Fragment of a Ritual for Singer,0.952630,0
P336141,Rituals on Shebat 23-25,0.944188,0
...,...,...,...
P238349,Restoring the Privileges of Nemed-Lagudu,0.979058,9
P313432,More on the Conspiracy of Sasi,0.977818,9
P334315,Prophylactic Rituals for the Royal Family,0.977492,9
P334746,Plea to the King to Cure Sickness,0.976309,9


# Renumber Topics
Rename the topics (columns) to start with 1, in accordance with the pyLDAvis visualization.

In [19]:
topics = [i+1 for i in range(ntopics)]
d_t_df.columns = topics
d_t_df

,1,2,3,4,5,6,7,8,9,10
P224378,0.009094,0.009094,0.009096,0.009096,0.009097,0.102053,0.009094,0.009095,0.009094,0.825185
P224382,0.044935,0.003866,0.046453,0.003866,0.003867,0.003866,0.087804,0.099572,0.003866,0.701906
P224383,0.003334,0.003334,0.320284,0.003334,0.003334,0.003335,0.003334,0.003334,0.003335,0.653042
P224386,0.006667,0.006667,0.006668,0.006668,0.006669,0.006667,0.006667,0.006669,0.156555,0.790102
P224388,0.006669,0.006671,0.098229,0.006670,0.006670,0.006670,0.006670,0.006670,0.006670,0.848412
...,...,...,...,...,...,...,...,...,...,...
X900011,0.002942,0.002943,0.002942,0.002942,0.002942,0.002942,0.245046,0.002942,0.373325,0.361034
X900012,0.002084,0.002084,0.477666,0.002084,0.002084,0.002084,0.002084,0.002084,0.053267,0.454479
X900013,0.001164,0.001163,0.001164,0.001163,0.001164,0.001164,0.001164,0.001163,0.014563,0.976129
X900014,0.002273,0.002273,0.979539,0.002273,0.002273,0.002274,0.002273,0.002273,0.002274,0.002274


# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [20]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of list. Each topic is represented by a list of tuples in the form `(word, probability)`. The following code pulls out the probabilities for each word in each topic (`topic_term[i][1]`) and creates a list of DataFrames with the words as index (rows) and the probabilities as the only column. The DataFrames are concatenated to a single DataFrame. 

In [34]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0) for i in range(0, ntopics)]
t_t_df_ = pd.concat(topic_term_list, axis=1, ignore_index=True)
t_t_df_.head()

/Users/veldhuis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,0,1,2,3,4,5,6,7,8,9
aban-bāšti[(a-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004
aban-lamassi[(a-precious-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000404,0.000014,0.000029,0.000004
aban-râmi['love'-stone]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004
abati[(meaning-unknown)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000029,0.000028
abašmû[(a-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004


Rename the columns to start with 1, and Transpose to Topic/Term matrix.

In [22]:
t_t_df_.columns = topics
t_t_df = t_t_df_.T
t_t_df

,aban-bāšti[(a-stone)]n,aban-lamassi[(a-precious-stone)]n,aban-râmi['love'-stone]n,abati[(meaning-unknown)]n,abašmû[(a-stone)]n,abbušu[(meaning-unknown)]n,abbūtu[fatherhood]n,abiktu[defeat]n,abku[captive]n,ablu[brought]aj,...,ṭēmūtu[of-order]n,ṭīdu[clay]n,ṭīmu[yarn]n,ṭīpu[addition]n,ṭīru[impression]n,ṭūbtu[peace]n,ṭūbu[goodness]n,ṭūbātu[happiness]n,ṭūdu[way]n,ṭūru[opopanax]n
1,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.004479,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
2,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000323,0.000014,0.000014,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014
3,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000296,0.000014,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014
4,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000119,0.000005,0.000005,...,0.000005,0.000005,0.000005,0.000094,0.000005,0.000005,0.000345,0.000549,0.000005,0.000005
5,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000410,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000159,0.000009,0.000009,0.000009,0.000009
6,0.000009,0.000009,0.000009,0.000009,0.000009,0.000070,0.000009,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000299,0.000009
7,0.000008,0.000404,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
8,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000108,...,0.000014,0.000014,0.000014,0.000014,0.000111,0.000014,0.001660,0.000014,0.000014,0.000014
9,0.000498,0.000029,0.000498,0.000029,0.000498,0.000029,0.000029,0.000029,0.000029,0.000029,...,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029,0.000029
10,0.000004,0.000004,0.000004,0.000028,0.000004,0.000004,0.000692,0.000329,0.000004,0.000004,...,0.000078,0.000004,0.000142,0.000004,0.000004,0.000004,0.005634,0.000004,0.000004,0.000061


In [23]:
#just checking
t_t_df['ēkallu[palace]n']

1     0.004932
2     0.006496
3     0.019528
4     0.000532
5     0.000031
6     0.004552
7     0.003679
8     0.003392
9     0.004021
10    0.009725
Name: ēkallu[palace]n, dtype: float64

# TODO
Export data for viz. in 5.3

In [32]:
topic_model = {'dictionary': dictionary,
                  'corpus' : corpus,
                  'ldamodel' : ldamodel,
                  't_t_df' : t_t_df,
                  'd_t_df' : d_t_df,
               'df' : df,
              'ntopics' : ntopics,
              'texts' : texts}

In [33]:
with open('output/topic_model.p', 'wb') as w:
    pickle.dump(topic_model, w)